In [ ]:
#!pip install selenium
#!pip install webdriver_manager
from selenium import webdriver

import os
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import codecs
from selenium.webdriver.common.keys import Keys
import re

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select

In [ ]:
website = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'
driver = webdriver.Chrome()
driver.get(website)

In [ ]:
next_page = driver.find_element(By.LINK_TEXT, 'Next »')
for x in driver.find_elements(By.XPATH,'//*[@id="main"]/div/div[4]/span[1]'):
    end = int(x.text.split(' ')[2].replace(',',''))
rang = []
film = []
jahr = []
fsk = []
dauer = []
genre = []
bewertung = []
regisseur = []
stars = []


while next_page:
    headers = driver.find_elements(By.TAG_NAME, 'h3')
    for head in headers:
        rang.append(head.text.split(' ')[0])
        jahr.append(head.text.split(' ')[-1])
        film.append(head.text.split(' ')[1:-1])
    rang = rang[:-1]
    jahr = jahr[:-1]
    film = film[:-1]
    genre_infos = driver.find_elements(By.XPATH, '//p[@class="text-muted "]')

    for genre_info in genre_infos:
        if len(genre_info.text.split('|')) == 3:
            fsk.append(genre_info.text.split('|')[0])
            dauer.append(genre_info.text.split('|')[1])
            genre.append(genre_info.text.split('|')[2])
        elif len(genre_info.text.split('|')) == 2:
            fsk.append('NaN')
            dauer.append(genre_info.text.split('|')[0])
            genre.append(genre_info.text.split('|')[1])
    ratings_bar =  driver.find_elements(By.CLASS_NAME, 'ratings-bar')

    for rating_bar in ratings_bar:
        bewertung.append(rating_bar.text.split(' ')[0])

    find_directors = driver.find_elements(By.XPATH, '//p[contains(text(), "Director")]/a')

    for director in find_directors:
        regisseur.append(director.text)

    find_stars = driver.find_elements(By.XPATH, '//p[contains(text(), "Stars")]/a')

    for star in find_stars:
        stars.append(star.text)

    if len(rang)== end:
        break
    next_page = driver.find_element(By.LINK_TEXT, 'Next »')
    next_page.click()

driver.quit()

In [ ]:

# lists to dataframe
import pandas as pd
df = pd.DataFrame()
df['rang'] = rang
df['film'] = film
df['jahr'] = jahr
df['fsk'] = fsk
df['dauer'] = dauer
df['genre'] = genre
df['bewertung'] = bewertung
df['regisseur'] = regisseur
df['stars'] = stars
df